## Comparison between initialization (LDA-DFT) with reconstruction

In [ ]:
import warnings as wn
wn.filterwarnings("ignore")

import os
import fuller
from natsort import natsorted
import glob as g
import numpy as np
import scipy.io as sio
from mpes import analysis as aly
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import interpolate
%matplotlib inline

In [ ]:
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = 'Arial'
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

colornames = ['#646464', '#666666', '#6a6a6a', '#6f6f6f', '#737373', '#787878', '#7d7d7d', '#828282', '#878787', '#8d8d8d', '#929292', '#989898', '#9e9e9e', '#a4a4a4', '#aaaaaa', '#b0b0b0', '#b6b6b6', '#bcbcbc', '#c2c2c2', '#c9c9c9', '#cfcfcf', '#d6d6d6', '#dcdcdc', '#e3e3e3', '#eaeaea', '#efefee', '#efeee5', '#efeddc', '#efecd3', '#eeebca', '#eeeac0', '#eee9b7', '#eee8ad', '#ede7a4', '#ede69a', '#ede590', '#ede487', '#ece37d', '#ece273', '#ece069', '#ecdf5f', '#ebde55', '#ebdd4b', '#ebdc41', '#ebdb37', '#ebd333', '#ebc933', '#ecbe32', '#ecb432', '#eda931', '#ee9e31', '#ee9330', '#ef8830', '#ef7d2f', '#f0722f', '#f0672e', '#f15c2e', '#f2512d', '#f2462d', '#f33b2c', '#f3302c', '#f4252b', '#f4192b', '#ef182f', '#e81834', '#e21939', '#db1a3e', '#d51a43', '#ce1b48', '#c71b4d', '#c11c52', '#ba1c58', '#b31d5d', '#ac1d62', '#a61e67', '#9f1e6c', '#981f72', '#911f77', '#8a207c', '#842182']
custom_cmap = mpl.colors.LinearSegmentedColormap.from_list('custom', colornames, N=256)

# Create plot folder if needed
if not os.path.exists('../results/figures'):
    os.mkdir('../results/figures')

In [ ]:
# Load energy bands from reconstrucion
recbands = np.load(r'../data/processed/wse2_recon/postproc_refrotsym_bands_lda.npy')
reconfile = r'../data/processed/wse2_recon/lda_recon_mrf_rec_00.h5'
einband = fuller.utils.loadHDF(reconfile, groups=['axes'])
kx, ky = einband['kx'], einband['ky']
nkx, nky = kx.size, ky.size

# Load energy bands from theory (used as initialization for the reconstruction)
theory = sio.loadmat(r'../data/theory/WSe2_LDA_bands.mat')
bslda = theory['evb']
kxxth, kyyth = theory['kxxsc'], theory['kyysc']
kxth, kyth = kxxth[:,0], kyyth[0,:]

In [ ]:
# Calculate the path for the auxiliary lines on figure
verts = np.array([[ 80.2399401 , 208.0445881 ],
       [175.36646777, 208.04459675],
       [221.9318386 , 126.24592903],
       [175.36871023,  44.44731539],
       [ 80.23765786,  44.44726939],
       [ 33.69597945, 126.24588838]])
pks = np.vstack((verts, verts[0,:]))
kxip = interpolate.interp1d(np.arange(0, 256), kx)
kyip = interpolate.interp1d(np.arange(0, 256), ky)
kpks = np.asarray([[kxip(i)-0.04, kyip(j)] for (i, j) in pks])
msk = aly.circmask(recbands[0,...], 128, 125, 105, sign='xnan', method='algebraic')

### Main Figure 2d

In [ ]:
ksc = 1.1 # Hyperparameter used in reconstruction to align the theory and data to same resolution
kxshift = 0.05 # Corrections for imperfect axis calibration
f, axs = plt.subplots(7, 2, figsize=(8.3, 30))
for i, ind in enumerate(range(0, 7)):
    
    intFunc = interpolate.RegularGridInterpolator((kx/ksc, ky/ksc), bslda[2*ind,33:-30, 41:-42],
                                                  bounds_error=False, fill_value=None)
    kxx, kyy = np.meshgrid(kx, ky, indexing='ij')
    kxx = np.reshape(kxx, (nkx * nky,))
    kyy = np.reshape(kyy, (nkx * nky,))
    Ek_scaled = np.reshape(intFunc(np.column_stack((kxx, kyy))), (nkx, nky))
    
    axs[i,0].imshow(Ek_scaled.T*msk, cmap=custom_cmap,
                    extent=[kx[0]+kxshift, kx[-1]+kxshift, ky[0], ky[-1]])
    axs[i,0].spines['right'].set_color('none')
    axs[i,0].spines['top'].set_color('none')
    axs[i,0].tick_params(axis='both', length=8, width=2, labelsize=28)
    axs[i,0].set_xlabel('$k_x \, (\mathrm{\AA}^{-1})$', fontsize=20) #, transform=axs[i,0].transAxes
    axs[i,0].xaxis.set_label_coords(0.88, 0.15)
    axs[i,0].set_ylabel('$k_y \, (\mathrm{\AA}^{-1})$', fontsize=20, rotation=0)
    axs[i,0].yaxis.set_label_coords(0.20, 0.90)
    if i < 6:
        axs[i,0].set_xticklabels('')
    axs[i,0].text(0.85, 0.93, '# '+str(ind+1), transform=axs[i,0].transAxes, fontsize=20)
    axs[i,0].axvline(x=0, ls='--', lw=1, color='k', dashes=[5, 4])
    axs[i,0].axhline(y=0, ls='--', lw=1, color='k', dashes=[5, 4])
    
    # Draw hexagon
    axs[i,0].plot(kpks[:,1]-0.02, kpks[:,0], '--k', lw=1, dashes=[5, 4])
    
    # Draw the reconstruction side
    axs[i,1].imshow(recbands[ind,...]*msk, cmap=custom_cmap, extent=[kx[0]+kxshift, kx[-1]+kxshift, ky[0], ky[-1]])
    axs[i,1].spines['right'].set_color('none')
    axs[i,1].spines['top'].set_color('none')
    axs[i,1].tick_params(axis='both', length=8, width=2, labelsize=28)
    axs[i,1].set_xlabel('$k_x \, (\mathrm{\AA}^{-1})$', fontsize=20)
    axs[i,1].xaxis.set_label_coords(0.88, 0.15)
    axs[i,1].set_ylabel('$k_y \, (\mathrm{\AA}^{-1})$', fontsize=20, rotation=0)
    axs[i,1].yaxis.set_label_coords(0.20, 0.90)
    if i < 6:
        axs[i,1].set_xticklabels('')
    axs[i,1].set_yticklabels('')
    axs[i,1].text(0.85, 0.93, '# '+str(ind+1), transform=axs[i,1].transAxes, fontsize=20)
    axs[i,1].axvline(x=0, ls='--', lw=1, color='k', dashes=[5, 4])
    axs[i,1].axhline(y=0, ls='--', lw=1, color='k', dashes=[5, 4])
    
    # Draw hexagon
    axs[i,1].plot(kpks[:,1]-0.01, kpks[:,0], '--k', lw=1, dashes=[5, 4])
    
    
axs[0,0]. set_title('Theory', fontsize=20, y=1.05)
axs[0,1]. set_title('Reconstruction', fontsize=20, y=1.05)
plt.subplots_adjust(wspace=0.08, hspace=0.08)
plt.savefig('../results/figures/fig_2d1.png', bbox_inches='tight', transparent=False, dpi=300)

In [ ]:
ksc = 1.1 # Hyperparameter used in reconstruction to align the theory and data to same resolution
f, axs = plt.subplots(7, 2, figsize=(8.3, 30))
for i, ind in enumerate(range(7, 14)):
    
    intFunc = interpolate.RegularGridInterpolator((kx/ksc, ky/ksc), bslda[2*ind,33:-30, 41:-42],
                                                  bounds_error=False, fill_value=None)
    kxx, kyy = np.meshgrid(kx, ky, indexing='ij')
    kxx = np.reshape(kxx, (nkx * nky,))
    kyy = np.reshape(kyy, (nkx * nky,))
    Ek_scaled = np.reshape(intFunc(np.column_stack((kxx, kyy))), (nkx, nky))
    
    axs[i,0].imshow(Ek_scaled.T*msk, cmap=custom_cmap,
                    extent=[kx[0]+0.05, kx[-1]+0.05, ky[0], ky[-1]])
    axs[i,0].spines['right'].set_color('none')
    axs[i,0].spines['top'].set_color('none')
    axs[i,0].tick_params(axis='both', length=8, width=2, labelsize=28)
    axs[i,0].set_xlabel('$k_x \, (\mathrm{\AA}^{-1})$', fontsize=20) #, transform=axs[i,0].transAxes
    axs[i,0].xaxis.set_label_coords(0.88, 0.15)
    axs[i,0].set_ylabel('$k_y \, (\mathrm{\AA}^{-1})$', fontsize=20, rotation=0)
    axs[i,0].yaxis.set_label_coords(0.20, 0.90)
    if i < 6:
        axs[i,0].set_xticklabels('')
    axs[i,0].text(0.85, 0.93, '# '+str(ind+1), transform=axs[i,0].transAxes, fontsize=20)
    axs[i,0].axvline(x=0, ls='--', lw=1, color='k', dashes=[5, 4])
    axs[i,0].axhline(y=0, ls='--', lw=1, color='k', dashes=[5, 4])
    
    # Draw hexagon
    axs[i,0].plot(kpks[:,1]-0.02, kpks[:,0], '--k', lw=1, dashes=[5, 4])
    
    # Draw the reconstruction side
    axs[i,1].imshow(recbands[ind,...]*msk, cmap=custom_cmap, extent=[kx[0]+kxshift, kx[-1]+kxshift, ky[0], ky[-1]])
    axs[i,1].spines['right'].set_color('none')
    axs[i,1].spines['top'].set_color('none')
    axs[i,1].tick_params(axis='both', length=8, width=2, labelsize=28)
    axs[i,1].set_xlabel('$k_x \, (\mathrm{\AA}^{-1})$', fontsize=20)
    axs[i,1].xaxis.set_label_coords(0.88, 0.15)
    axs[i,1].set_ylabel('$k_y \, (\mathrm{\AA}^{-1})$', fontsize=20, rotation=0)
    axs[i,1].yaxis.set_label_coords(0.20, 0.90)
    if i < 6:
        axs[i,1].set_xticklabels('')
    axs[i,1].set_yticklabels('')
    axs[i,1].text(0.85, 0.93, '# '+str(ind+1), transform=axs[i,1].transAxes, fontsize=20)
    axs[i,1].axvline(x=0, ls='--', lw=1, color='k', dashes=[5, 4])
    axs[i,1].axhline(y=0, ls='--', lw=1, color='k', dashes=[5, 4])
    
    # Draw hexagon
    axs[i,1].plot(kpks[:,1]-0.01, kpks[:,0], '--k', lw=1, dashes=[5, 4])
    
    
axs[0,0]. set_title('Theory', fontsize=20, y=1.05)
axs[0,1]. set_title('Reconstruction', fontsize=20, y=1.05)
plt.subplots_adjust(wspace=0.08, hspace=0.08)
plt.savefig('../results/figures/fig_2d2.png', bbox_inches='tight', transparent=False, dpi=300)